In [62]:
#### GIS PROGRAMMING TO FIND A LOCATION FOR A NEW ASTRONOMICAL OBSERVATORY IN FRANCE ####
#### Sannier Maelle ####
#### 04.30.2020 ####
#### EPF Graduate School of Engineering, Sceaux, France - RUDN, Moscow, Russia ####

######## Parameters ######### 
# workspace = storage space defined by the user e.g "C:/Observatories"
# fc1 = first layer to use to create buffers : french cities
# b1  = Radius Buffer in Kilometers for 1rst feature class
# fc2 = second layer to use to create buffers : existing observatories
# b2  = Radius buffer in kilometers for 2nd feature class
# citysize = We want to avoid cities over citysize inhabitants
# coef1 = coefficient in the weighted sum for Crit1
# coef2 = coefficient in the weighted sum for Crit2
# coef3 = coefficient in the weighted sum for Crit3
# percentage = percentage of best pixels of the weithed sum layer that the user wants to keep

def Find_Observatories(workspace,fc1,b1,fc2,b2,citysize,coef1,coef2,coef3,percentage):
    
        ######## Initialization ######### 
        import arcpy
        workspace = arcpy.env.workspace
        arcpy.env.overwriteOutput = True

        intermediate_layers=["fc1_buffer","fc2_buffer","Buffers_Union","Buffers_Dissolved",\
                             "Lum_Vector","Crit1VctEra","Crit1Rst","Crit1NormInv","Crit1Norm",\
                            "RelcassAlt_Vct","Crit2VctEra","Crit2Rst","Crit2Norm",\
                            "Clouds_Vector","Crit3VctEra","Crit3Rst","Crit3Norm",\
                            "arithmeticlayer","Bestpixels","Finalcities","BestpixelsVct","FinalResult"]
                             
        for intermediate_layer in intermediate_layers:
            if arcpy.Exists(intermediate_layer):
                arcpy.Delete_management(intermediate_layer)
                
        ######## Verification #########     
        
        if percentage > 100 or percentage <0 :
            print("The percentage must be between 0 and 100")
        
                
        ######## Buffers layers #########         
                
        #Creation of 2 buffers analysis around the cities over 50k inhabitant 
        #and existing astronomical observatories
        print("Creating buffer around cities with more than",citysize,"inhabitants...","fc1:",fc1)
        outselection = arcpy.SelectLayerByAttribute_management(fc1,'NEW SELECTION', "PTOT >"+str(citysize))
        arcpy.Buffer_analysis(outselection, workspace +"/French_cities" + "/"+ "fc1_buffer.shp",b1)
        arcpy.Buffer_analysis(fc2, workspace + "/Observatories" +"/"+ "fc2_buffer.shp",b2)
        print("First feature class: ",fc1,"with ",b1,"buffer",\
              "\nSecond feature class: ",fc2, "with ",b2,"buffer")
        
        #Union and Dissolve of the 2 buffers analysis
        arcpy.Union_analysis(["fc1_buffer","fc2_buffer"],"Buffers_Union")
        arcpy.Dissolve_management("Buffers_Union","Buffers_Dissolved")
        
        
        ######## First Criteria : Luminous Pollution ########
        
        #Conversion of the vector into a Raster 
        #to be able to perfom Erase analysis with the Buffer Union layer
        #And finnaly convert it again into a Raster
        #arcpy.RasterToPolygon_conversion("Lum_in_France.tif", workspace + "/" + "Lum_Vector","SIMPLIFY","Value")
        arcpy.Erase_analysis(workspace + "/Luminous_Pollution" + "/" + "Lum_Vector.shp","Buffers_Dissolved.shp","Crit1VctEra")
        arcpy.FeatureToRaster_conversion("Crit1VctEra.shp","gridcode", workspace + "/" + "Crit1Rst",0.01)

        #Normalization of the layer into a 0-100 range
        luminouslayer = arcpy.Raster(workspace + "/Luminous_Pollution" + "/" + "Crit1Rst")
        luminousNormalized = (luminouslayer - luminouslayer.minimum) / (luminouslayer.maximum - luminouslayer.minimum) *100
        luminousNormalized.save(workspace + "/Luminous_Pollution" + "/" + "Crit1NormInv.tif")
        
        #Inversion of the layer so that values close to 100 represent the most wanted areas
        luminousInv = (luminousNormalized - luminousNormalized.maximum)* -1
        luminousInv.save(workspace + "/Luminous_Pollution" + "/" + "Crit1Norm.tif")
        
        
        ######## Second Criteria : Altitude ########
        
        #ReClassification of the layer to reduce the calculation time
        #Conversion of the vector into a Raster 
        #Erase analysis with the Buffer Union layer
        #And finnaly convert it again into a Raster
        #outClassifyAlt= arcpy.sa.Reclassify("Altitude_in_France.crf", "Value", \
                        #arcpy.sa.RemapRange([[0,225,1],[225,563,2],[563,1070,3],[1070,1821,4],[1821,4788,5]]))
        #outClassifyAlt.save(workspace + "/" + "Crit2Class.tif")
        #arcpy.RasterToPolygon_conversion("Crit2Class.tif", workspace + "/" + "RelcassAlt_Vct","SIMPLIFY","Value")
        arcpy.Erase_analysis(workspace +"/Altitude" + "/"+ "RelcassAlt_Vct.shp","Buffers_Dissolved.shp","Crit2VctEra")
        arcpy.FeatureToRaster_conversion("Crit2VctEra.shp","gridcode", workspace + "/Altitude" +"/" + "Crit2Rst",0.01)
        
        #Normalization of the layer into a 0-100 range
        altitudelayer = arcpy.Raster(workspace + "/Altitude" +"/" + "Crit2Rst")
        altitudeNormalized = (altitudelayer - altitudelayer.minimum) / (altitudelayer.maximum-altitudelayer.minimum) *100
        altitudeNormalized.save(workspace + "/Altitude" +"/" + "Crit2Norm.tif")
        
        
        ######## Third Criteria : Cloudiness ########
        
        #Conversion of the vector into a Raster 
        #Erase analysis with the Buffer Union layer
        #And finnaly convert it again into a Raster
        #arcpy.RasterToPolygon_conversion("Clouds_in_France",workspace + "/" + "Clouds_Vector","SIMPLIFY","Value")
        arcpy.Erase_analysis(workspace + "/Cloudiness" +"/"+ "Clouds_Vector.shp","Buffers_Dissolved.shp","Crit3VctEra")
        arcpy.FeatureToRaster_conversion("Crit3VctEra.shp","gridcode",workspace +"/Cloudiness" + "/" + "Crit3Rst",0.01)
        
        #Normalization of the layer into a 0-100 range
        cloudslayer = arcpy.Raster(workspace +"/Cloudiness" + "/" + "Crit3Rst")
        cloudsNormalized = (cloudslayer - cloudslayer.minimum) / (cloudslayer.maximum - cloudslayer.minimum) *100
        cloudsNormalized.save(workspace + "/Cloudiness" +"/" + "Crit3Norm.tif")

        
        ######## Arithmeric Layer : weighted sum ########
        
        arithmeticlayer = coef1 * Crit1Norm + coef2 * Crit2Norm + coef3 * Crit3Norm
        arithmeticlayer.save(workspace + "/Results" +"/" + "arithmeticlayer.tif")
        print("Coefficients for weighted sum: ",\
              "\nCoeffient 1 :",coef1,\
              "\nCoeffient 2 :",coef2,\
              "\nCoeffient 3 :",coef3)
        
        ######## Final Results ########
        
        #Definition of the percentage of best pixels the user want to keep
        #Then finding the minumum value of the pixels that the user want to keep
        condition = (arithmeticlayer.maximum - arithmeticlayer.minimum ) / (100/percentage)
        treshold = arithmeticlayer.maximum - condition
        print("Percentage of best pixels kept: ", percentage,"%")
        
        #Creation of the new layer of Best pixels
        outcon = arcpy.sa.Con(arithmeticlayer,1,"","Value >" + str(treshold))
        outcon.save(workspace +  "/Results" +"/" + "Bestpixels")
    
        arcpy.Erase_analysis("frenchville.shp","Buffers_Dissolved.shp","Finalcities")
        
        #Final results giving a village in output 
        arcpy.RasterToPolygon_conversion("Bestpixels","BestpixelsVct.shp","SIMPLIFY","VALUE")
        arcpy.Intersect_analysis(["BestpixelsVct.shp","Finalcities.shp"],"FinalResult.shp")
        
        layerFields = ["*"]
        nbr_results = arcpy.GetCount_management(workspace +  "/Results" +"/" + "FinalResult.shp")
        print("Nbr de resultat: ", nbr_results)
        with arcpy.da.SearchCursor(workspace +  "/Results" +"/" + "FinalResult.shp", layerFields) as cursor:
            for row in cursor:
                print("Cities: \n","Name :",row[6],",", "GPS Coordinates: ",\
                      "( latitude:",row[11],"longitude:",row[12],")","\n")
        